In [69]:
import requests
from bs4 import BeautifulSoup
from afinn import Afinn
import pandas as pd

# Step 1: Fetch the first page of results
url = "https://www.commonsensemedia.org/search/kids-books-about-911"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

review_teasers = soup.find_all('p', class_='review-teaser-one-liner')
reviews = [teaser.get_text(strip=True) for teaser in review_teasers]

# Step 3: Sentiment analysis using Afinn
afinn = Afinn()

# Create a DataFrame
df = pd.DataFrame(reviews, columns=['reviews'])

# Calculate the sentiment score and label
df['afn_score'] = df['reviews'].apply(afinn.score)
df['sentiment'] = df['afn_score'].apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'neutral'))

# Display the resulting DataFrame
print(df)


   reviews                                                        afn_score sentiment
0       Colorful, fascinating portraits of 50 kids in 50 states.  3.0        positive
1   Baseball, broken hearts in decent 2nd chance romance sequel.  3.0        positive
2       Wacky family dynamics and potty humor in 19th Wimpy Kid.  2.0        positive
3   Exciting misadventures at Percy's pet-sitting gig gone wild.  3.0        positive
4   Inspired narrative, world-building in luminous series debut.  2.0        positive
5    Educational special is kid-friendly view of U.S. elections.  2.0        positive
6      Charming graphic memoir of a reluctant move to Hong Kong.  3.0        positive
7    Strong girl lead in Hmong folklore inspired fantasy sequel.  4.0        positive
8       Sweet, poignant, funny tale of magic, family torn apart.  6.0        positive
9         Teen riders rebel in exciting and dark unicorn series.  1.0        positive
10       Gripping, violent, sensual dark academia vamp

### 4

In [71]:
import requests
from bs4 import BeautifulSoup
from afinn import Afinn
import pandas as pd

# Step 1: Fetch the first page of results
url = "https://www.commonsensemedia.org/search/kids-books-about-911"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Step 2: Extract reviews and book titles
review_teasers = [teaser.get_text(strip=True) for teaser in soup.find_all('p', class_='review-teaser-one-liner')]
titles = [title.get_text(strip=True) for title in soup.find_all('h3', class_='review-teaser-title')]

# Ensure the lengths of the lists are consistent
min_length = min(len(review_teasers), len(titles))
review_teasers = review_teasers[:min_length]
titles = titles[:min_length]

# Step 3: Sentiment analysis using Afinn
afinn = Afinn()

# Create a DataFrame
df = pd.DataFrame({
    'book_title': titles,
    'reviews': review_teasers
})

# Calculate the sentiment score and label
df['afn_score'] = df['reviews'].apply(afinn.score)
df['sentiment'] = df['afn_score'].apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'neutral'))

# Display adjustments for better formatting
pd.set_option('display.max_columns', None)   # Ensure all columns are displayed
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to new lines
df['book_title'] = df['book_title'].str.slice(0, 50)  # Limit book_title column to 50 characters

# Display the resulting DataFrame
print(df)


   book_title                                          reviews                                                        afn_score sentiment
0   All About U.S.: A Look at the Lives of 50 Real Kid      Colorful, fascinating portraits of 50 kids in 50 states.  3.0        positive
1   Nothing Like the Movies: Better Than the Movies, B  Baseball, broken hearts in decent 2nd chance romance sequel.  3.0        positive
2                       Diary of a Wimpy Kid: Hot Mess      Wacky family dynamics and potty humor in 19th Wimpy Kid.  2.0        positive
3   Wrath of the Triple Goddess: Percy Jackson and the  Exciting misadventures at Percy's pet-sitting gig gone wild.  3.0        positive
4   Impossible Creatures: Impossible Creatures, Book 1  Inspired narrative, world-building in luminous series debut.  2.0        positive
5                              Kids Pick the President   Educational special is kid-friendly view of U.S. elections.  2.0        positive
6   Uprooted: A Memoir About What 

### 5

In [53]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [97]:
import requests
from bs4 import BeautifulSoup
from afinn import Afinn
from nrclex import NRCLex
import pandas as pd
from IPython.core.display import display, HTML

# Fetch and process data (same as before)
url = "https://www.commonsensemedia.org/search/kids-books-about-911"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
review_teasers = [teaser.get_text(strip=True) for teaser in soup.find_all('p', class_='review-teaser-one-liner')]
titles = [title.get_text(strip=True) for title in soup.find_all('h3', class_='review-teaser-title')]

min_length = min(len(review_teasers), len(titles))
review_teasers = review_teasers[:min_length]
titles = titles[:min_length]
afinn = Afinn()

def extract_emotions(text):
    emotion_analysis = NRCLex(text)
    return emotion_analysis.top_emotions

df = pd.DataFrame({
    'book_title': titles,
    'reviews': review_teasers
})

df['afn_score'] = df['reviews'].apply(afinn.score)
df['sentiment'] = df['afn_score'].apply(lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'neutral'))
df['top_emotions'] = df['reviews'].apply(extract_emotions)

# Save each review and top emotions to a text file
with open('nrclex_analysis_output.txt', 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        f.write(f"Review: {row['reviews']}\n")
        f.write(f"Top Emotions: {row['top_emotions']}\n")
        f.write("\n")

# Adjust display format for Jupyter Notebook
book_title_max_len = 30
reviews_max_len = 40
top_emotions_max_len = 150

df['book_title'] = df['book_title'].apply(lambda x: (x[:book_title_max_len] + '..') if len(x) > book_title_max_len else x)
df['reviews'] = df['reviews'].apply(lambda x: (x[:reviews_max_len] + '..') if len(x) > reviews_max_len else x)
df['top_emotions'] = df['top_emotions'].apply(lambda x: (str(x)[:top_emotions_max_len] + '..') if len(str(x)) > top_emotions_max_len else x)

# Display DataFrame with horizontal scrollbar using HTML in Jupyter Notebook
display(HTML(df.to_html().replace('<table', '<table style="display:block; overflow-x:auto; white-space:nowrap;"')))


C:\Users\nikhi\AppData\Local\Temp\ipykernel_28668\3777929470.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,book_title,reviews,afn_score,sentiment,top_emotions
0,All About U.S.: A Look at the ..,"Colorful, fascinating portraits of 50 ki..",3.0,positive,"[(positive, 1.0)]"
1,Nothing Like the Movies: Bette..,"Baseball, broken hearts in decent 2nd ch..",3.0,positive,"[('fear', 0.14285714285714285), ('surprise', 0.14285714285714285), ('positive', 0.14285714285714285), ('sadness', 0.14285714285714285), ('anticipation.."
2,Diary of a Wimpy Kid: Hot Mess,Wacky family dynamics and potty humor in..,2.0,positive,"[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust.."
3,Wrath of the Triple Goddess: P..,Exciting misadventures at Percy's pet-si..,3.0,positive,"[(surprise, 0.5), (negative, 0.5)]"
4,Impossible Creatures: Impossib..,"Inspired narrative, world-building in lu..",2.0,positive,"[(trust, 1.0)]"
5,Kids Pick the President,Educational special is kid-friendly view..,2.0,positive,"[(positive, 0.5), (joy, 0.5)]"
6,Uprooted: A Memoir About What ..,Charming graphic memoir of a reluctant m..,3.0,positive,"[(fear, 0.5), (negative, 0.5)]"
7,"Pahua and the Dragon's Secret,..",Strong girl lead in Hmong folklore inspi..,4.0,positive,"[(positive, 0.3333333333333333)]"
8,The Hotel Balzaar: Norendy Tal..,"Sweet, poignant, funny tale of magic, fa..",6.0,positive,"[(positive, 0.5), (negative, 0.5)]"
9,Skandar and the Skeleton Curse..,Teen riders rebel in exciting and dark u..,1.0,positive,"[('fear', 0.1111111111111111), ('anger', 0.1111111111111111), ('trust', 0.1111111111111111), ('surprise', 0.1111111111111111), ('positive', 0.11111111.."
